In [1]:
'''
使用的数据可以从以下网址下载：
https://www.kaggle.com/c/dogs-vs-cats/data
在我们的设置中，我们：
- 创建一个 data/ 文件夹
- 在 data/ 里面 创建 train/ and validation/ 子文件夹
- 在 train/ 和 validation/ 里面 创建 cats/ and dogs/ 子文件夹
- 将猫图片索引0-999放入data/train/cats中
- 将猫图片索引1000-1400放在data/validation/cats中
- 把狗图片索引12500-13499放在data/train/dogs中
- 将狗图片索引13500-13900放在data/validation/dogs中
因此，我们每个类有1000个训练范例，每个类有400个验证范例。
总之，这是我们的目录结构：
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# 图像的维度。
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16


def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # 构建VGG16网络
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(train_data_dir,
                                            target_size=(img_width,
                                                         img_height),
                                            batch_size=batch_size,
                                            class_mode=None,
                                            shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
            bottleneck_features_train)

    generator = datagen.flow_from_directory(validation_data_dir,
                                            target_size=(img_width,
                                                         img_height),
                                            batch_size=batch_size,
                                            class_mode=None,
                                            shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
            bottleneck_features_validation)


def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array([0] * int((nb_train_samples / 2)) +
                            [1] * int((nb_train_samples / 2)))

    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array([0] * int((nb_validation_samples / 2)) +
                                 [1] * int((nb_validation_samples / 2)))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    model.fit(train_data,
              train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottlebeck_features()
train_top_model()

Using TensorFlow backend.
D:\ProgramData\Anaconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\ProgramData\Anaconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\ProgramData\Anaconda3\envs\deep\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\ProgramData\Anaconda3\envs\deep\lib\site-packag


Found 2000 images belonging to 2 classes.

Found 800 images belonging to 2 classes.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.6725 - accuracy: 0.7605 - val_loss: 0.4694 - val_accuracy: 0.7975
Epoch 2/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.3647 - accuracy: 0.8495 - val_loss: 0.2589 - val_accuracy: 0.9038
Epoch 3/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2969 - accuracy: 0.8885 - val_loss: 0.2560 - val_accuracy: 0.9038
Epoch 4/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2457 - accuracy: 0.8980 - val_loss: 0.2658 - val_accuracy: 0.9025
Epoch 5/50
2000/2000 [==============================] - 3s 2ms/step - loss: 0.2242 - accuracy: 0.9100 - val_loss: 0.3235 - val_accuracy: 0.8900
Epoch 6/50
2000/2000 [==============================] 